# PRV Assignment 1

- Pablo Garcia Fernández

Initial configuration

In [202]:
# Initial configuration
import numpy as np
from math import cos,sin

#######################################
## PREDIFINE CONSTATNS OF THE EXERCISE

ROLL=2.5; PITCH=5; YAW=60 
TRANSLATION = np.array([0, 0, 0.25])

#F = 3.62 # focal length (mm)
F = 2406.1 # focal length
CX = 25.498; CY=-33.0153 # principal point coordinates
K1 = 0.00247448; K2 = -0.0240244; K3 = 0.0604812; K4 = -0.0275491 # radial distorsion
P1 = 0.000150734; P2 = -0.000271032 # tangential distorsion
B1 = -5.67637; B2 = 0.339356 # affine

PIXEL_SIZE = 0.00156  # (mm)
W = 4000   # (px)
H = 3000   # (pX)

# W = RESOLUTION_W * PIXEL_SIZE  # (mm)
# H = RESOLUTION_H * PIXEL_SIZE  # (mm)
######################################

Auxiliar functions

In [203]:
def enu_to_ned_ned_to_enu():
    transformation_matrix = np.array([
        [0., 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, -1, 0],
		[0, 0, 0, 1]
    ])
    return transformation_matrix


def _3D_rotation_matrix(roll, pitch, yaw, order='ypr', inverso=False):
	# Return the rotation matrix in 3D coordinate systems
	# Assumming ISPRS moving axes
	yaw = np.array([
		[cos(yaw), sin(yaw), 0, 0],
		[-sin(yaw), cos(yaw), 0, 0],
		[0, 0, 1, 0],
		[0, 0, 0, 1]
	])
	pitch = np.array([
		[cos(pitch), 0, -sin(pitch), 0],
		[0, 1, 0, 0],
		[sin(pitch), 0, cos(pitch), 0],
		[0, 0, 0, 1]
	])
	roll = np.array([
		[1, 0, 0, 0],
		[0, cos(roll), sin(roll), 0],
		[0, -sin(roll), cos(roll), 0],
		[0, 0, 0, 1]
	]) 

	rotation_matrix = None
	
	if order=='ypr':
		if inverso:
			rotation_matrix = yaw.T @ pitch.T @ roll.T
		else:
			rotation_matrix = roll @ pitch @ yaw
	elif order == 'rpy':
		if inverso:
			rotation_matrix = roll.T @ pitch.T @ yaw.T
		else:
			rotation_matrix = yaw @ pitch @ roll
	else:
		print("Order not supported")

	return rotation_matrix

### World CS -> INS CS
### INS CS -> INS CS

In [204]:
def ins_to_wcs():
    # Note -> WCS is ENU
        
    # 1) yaw, pitch, roll rotation
    t1 = _3D_rotation_matrix(ROLL, PITCH, YAW, inverso=True)
    
    # 2) NED to ENU
    t2 = enu_to_ned_ned_to_enu()
    
    return t2 @ t1

def wcs_to_ins():
    # 1) ENU to NED
    t1 = enu_to_ned_ned_to_enu()

    # 2) yaw, pitch, roll rotation
    t2 = _3D_rotation_matrix(ROLL, PITCH, YAW, inverso=False)

    return t2 @ t1

### INS CS -> Gimbal CS
### Gimbal CS -> INS CS 

In [205]:
# They are the same
def gimbal_to_ins(traslation_vector):
    # 1) Translation + 2) ENU to NED
    transformation_matrix = enu_to_ned_ned_to_enu()
    transformation_matrix[:-1,-1] = traslation_vector

    return transformation_matrix

def ins_to_gimbal(traslation_vector):
    # 1) NED to ENU + 2) - translation
    transformation_matrix = enu_to_ned_ned_to_enu()
    transformation_matrix[:-1,-1] = traslation_vector

    return transformation_matrix


### Gimbal CS -> Camera CS
### Camera CS -> Gimbal CS

In [206]:
def camera_to_gimbal():
	# Frontal to nadiral
	# Rotation -90º axis X
	t = np.array([
		[1, 0, 0, 0],
		[0, 0, -1, 0],
		[0, 1, 0, 0],
		[0, 0, 0, 1]
	]) 
	return t

def gimbal_to_camera():
	# Nadiral to frontal
	# Rotation 90º axis X
	t = np.array([
		[1, 0, 0, 0],
		[0, 0, 1, 0],
		[0, -1, 0, 0],
		[0, 0, 0, 1]
	]) 
	return t

### Composition of all transformations

In [207]:
# 1st path: camera to wcs
t1 = camera_to_gimbal()
print("camera to gimbal:\n", t1)
t2 = gimbal_to_ins(TRANSLATION)
print("gimbal to ins:\n", t2)
t3 = ins_to_wcs()
print("ins to wcs:\n",t3)
# TOTAL
t = t3 @ t2 @ t1
print("full matrix:", t)
### Ejemplo de proyeccion
punto = np.array([3, 4, 2, 1])
proyeccion = t @ punto
print("Proyeccion del punto", punto, "a ", proyeccion)
print("\n\n")

# Camino 2 de wcs a camera
t1 = wcs_to_ins()
print("wcs to ins:\n", t1)
t2 = ins_to_gimbal(TRANSLATION)
print("ins to gimbal:\n", t2)
t3 = gimbal_to_camera()
print("gimbal to camera:\n",t3)
# TOTAL
t = t3 @ t2 @ t1
print("full matrix:", t)
### Ejemplo de proyeccion
punto = np.array([1.72742007, 4.87534516, 0.55635374, 1.   ])
proyeccion = t @ punto
print("Proyeccion del punto", punto, "a ", proyeccion)

camera to gimbal:
 [[ 1  0  0  0]
 [ 0  0 -1  0]
 [ 0  1  0  0]
 [ 0  0  0  1]]
gimbal to ins:
 [[ 0.    1.    0.    0.  ]
 [ 1.    0.    0.    0.  ]
 [ 0.    0.   -1.    0.25]
 [ 0.    0.    0.    1.  ]]
ins to wcs:
 [[-0.08646325  0.93794718  0.33582613  0.        ]
 [-0.27016355  0.30238269 -0.91409866  0.        ]
 [-0.95892427 -0.16976392  0.22725415  0.        ]
 [ 0.          0.          0.          1.        ]]
full matrix: [[ 0.93794718 -0.33582613  0.08646325  0.08395653]
 [ 0.30238269  0.91409866  0.27016355 -0.22852467]
 [-0.16976392 -0.22725415  0.95892427  0.05681354]
 [ 0.          0.          0.          1.        ]]
Proyeccion del punto [3 4 2 1] a  [1.72742007 4.87534516 0.55635374 1.        ]



wcs to ins:
 [[-0.08646325 -0.27016355 -0.95892427  0.        ]
 [ 0.93794718  0.30238269 -0.16976392  0.        ]
 [ 0.33582613 -0.91409866  0.22725415  0.        ]
 [ 0.          0.          0.          1.        ]]
ins to gimbal:
 [[ 0.    1.    0.    0.  ]
 [ 1.    0.    

## FRAME CAMERA MODEL

In [208]:
# CAMERA CS (3D x,y,z) -> IMAGE CS (2D u,v)
def camera_model(X,Y,Z):
    
    x = X/Z
    y = Y/Z
    r = np.sqrt(x**2 + y**2)
    
    x_prima = x*(1 + K1*r**2 + K2*r**4 + K3*r**6 + K4*r**8) + (P1*(r**2+2*x**2) + 2*P2*x*y)
    y_prima = y*(1 + K1*r**2 + K2*r**4 + K3*r**6 + K4*r**8) + (P2*(r**2+2*y**2) + 2*P1*x*y)
    
    u = W * 0.5 + CX + x_prima*F + x_prima*B1 + y_prima*B2
    v = H * 0.5 + CY + y_prima*F

    return u,v


# Prueba 
aa = camera_model(3, 5, 3)
print(aa)

(-2017.4691891178325, -5290.383167313029)


## Camera calibration-undistort

<img src="asd.jpeg" width="800">

In [209]:
# IMAGE -> CAMERA
def image_to_camera(u,v, threshold):
    """
    De momento hecho para 1 solo pixel
    """
    
    # Obtain estimation pixels
    y_prima = (v - 0.5*H - CY) / F  
    x_prima = (u - 0.5*W - CX - y_prima*B2) / (F + B1) 

    # Initialize estimated undistorted coordinates and error
    x = x_prima
    y = y_prima
    error = np.Inf
    
    while(error > threshold):
        r = np.sqrt(x**2 + y**2)
        
        tangential = (1 + K1*r**2 + K2*r**4 + K3*r**6 + K4*r**8)
        u_prima, v_prima = camera_model(x,y,1)

        dl_uprima = (u_prima - u) / np.sqrt((u - x)**2 + (v - y)**2)
        dl_vprima = (v_prima - v) / np.sqrt((u - x)**2 + (v - y)**2)
       
        duprima_x = F * (tangential + 4*P1*x + 2*P2*y) + B1 * (tangential + 4*P1*x + 2*P2*y) + B2*2*P1*y
        dvprima_x = F*2*P1*y
        duprima_y = B2 * (tangential + 4*P2*y + 2*P1*x) + F*2*P2*x + B1*2*P2*x
        dvprima_y = F * (tangential + 4*P2*y + 2*P1*x)

        x = x - 0.000001 * (dl_uprima*duprima_x + dl_vprima*dvprima_x)
        y = y - 0.000001 * (dl_uprima*duprima_y + dl_vprima*dvprima_y)
        
        u_prima, v_prima = camera_model(x,y,1)
        error = np.sqrt((u - u_prima)**2 + (v - v_prima)**2)
        print(f"x: {x}, y: {y}")
        print(error)


image_to_camera(10, 1, 0.1)




x: -0.833675261903863, y: -0.6047647717679293
14.406547609634504
x: -0.8311294373852657, y: -0.6028229299241763
6.21853404584042
x: -0.8300278311267794, y: -0.6019881034525193
2.6839553875514244
x: -0.8295511665887606, y: -0.6016293507984926
1.1583875665296721
x: -0.8293448954595978, y: -0.601475236282768
0.49996043451582806
x: -0.8292556215792793, y: -0.6014090531143288
0.21578736834431014
x: -0.82921697760481, y: -0.6013806410086203
0.09313829618016449
